In [1]:
import mysql.connector
import pandas as pd
import os
from openpyxl import Workbook

# SQL query
sql = """
SELECT encuestador ENCUESTADOR, depto DEPTO, mupio MUPIO, sector SECTOR, estructura ESTRUCTURA, vivienda VIVIENDA, hogar HOGAR, p.cp CP, 
	10 CAPITULO, 'C' SECCION, 2 PREGUNTA,
	'P10C02 Ocupación principal falta o insuficientemente descrita' AS 'DEFINICION DE INCONSISTENCIA',
	'10C00251' AS 'CODIGO ERROR', '' COMENTARIOS,
	p.p10c02 VALOR
FROM `level-1` l
	INNER JOIN cases c ON c.id=l.`case-id`
	INNER JOIN personas p ON p.`level-1-id` = l.`level-1-id` 
	INNER JOIN caratula r ON r.`level-1-id`=l.`level-1-id`
WHERE c.deleted=0 AND p.p10c01>=1 AND r.estado_pr=1 AND (LENGTH(IFNULL(p.p10c02,''))<=3 OR UPPER(IFNULL(p.p10c02,'')) IN ('NO','NA','N/A','NADA','ESTUDIA','ESTUDIAR','NINGUNO') ) AND 
	DATEDIFF(CURDATE(),(SELECT STR_TO_DATE(MAX(v.r1_fecha_inicial),'%d/%m/%y') FROM registro_de_visitas_pr v WHERE v.`level-1-id`=l.`level-1-id`))<=4
UNION
SELECT encuestador ENCUESTADOR, depto DEPTO, mupio MUPIO, sector SECTOR, estructura ESTRUCTURA, vivienda VIVIENDA, hogar HOGAR, p.cp CP, 
	10 CAPITULO, 'C' SECCION, 3 PREGUNTA,
	'P10C03 Actividad principal falta o insuficientemente descrita' AS 'DEFINICION DE INCONSISTENCIA',
	'10C00351' AS 'CODIGO ERROR', '' COMENTARIOS,
	p.p10c03 VALOR
FROM `level-1` l
	INNER JOIN cases c ON c.id=l.`case-id`
	INNER JOIN personas p ON p.`level-1-id` = l.`level-1-id` 
	INNER JOIN caratula r ON r.`level-1-id`=l.`level-1-id`
WHERE c.deleted=0 AND p.p10c01>=1 AND r.estado_pr=1 AND (LENGTH(IFNULL(p.p10c03,''))<=3 OR UPPER(IFNULL(p.p10c03,'')) IN ('NO','NA','N/A','NADA','ESTUDIA','ESTUDIAR','NINGUNA','NINGUNO') ) AND 
	DATEDIFF(CURDATE(),(SELECT STR_TO_DATE(MAX(v.r1_fecha_inicial),'%d/%m/%y') FROM registro_de_visitas_pr v WHERE v.`level-1-id`=l.`level-1-id`))<=4
UNION
SELECT encuestador ENCUESTADOR, depto DEPTO, mupio MUPIO, sector SECTOR, estructura ESTRUCTURA, vivienda VIVIENDA, hogar HOGAR, p.cp CP, 
	10 CAPITULO, 'C' SECCION, 4 PREGUNTA,
	'P10C04 Empresa falta o insuficientemente descrita' AS 'DEFINICION DE INCONSISTENCIA',
	'10C00451' AS 'CODIGO ERROR', '' COMENTARIOS,
	p.p10c04 VALOR
FROM `level-1` l
	INNER JOIN cases c ON c.id=l.`case-id`
	INNER JOIN personas p ON p.`level-1-id` = l.`level-1-id` 
	INNER JOIN caratula r ON r.`level-1-id`=l.`level-1-id`
WHERE c.deleted=0 AND p.p10c01>=1 AND r.estado_pr=1 AND (LENGTH(IFNULL(p.p10c04,''))<=2 OR UPPER(IFNULL(p.p10c04,'')) IN ('NO','NA','N/A','POR','NADA','ESTUDIA','ESTUDIAR') ) AND 
	DATEDIFF(CURDATE(),(SELECT STR_TO_DATE(MAX(v.r1_fecha_inicial),'%d/%m/%y') FROM registro_de_visitas_pr v WHERE v.`level-1-id`=l.`level-1-id`))<=4
UNION
SELECT encuestador ENCUESTADOR, depto DEPTO, mupio MUPIO, sector SECTOR, estructura ESTRUCTURA, vivienda VIVIENDA, hogar HOGAR, p.cp CP, 
	10 CAPITULO, 'C' SECCION, 7 PREGUNTA,
	'P10C07 Productos falta o insuficientemente descrita' AS 'DEFINICION DE INCONSISTENCIA',
	'10C00751' AS 'CODIGO ERROR', '' COMENTARIOS,
	p.p10c07 VALOR
FROM `level-1` l
	INNER JOIN cases c ON c.id=l.`case-id`
	INNER JOIN personas p ON p.`level-1-id` = l.`level-1-id` 
	INNER JOIN caratula r ON r.`level-1-id`=l.`level-1-id`
WHERE c.deleted=0 AND p.p10c01>=1 AND r.estado_pr=1 AND (LENGTH(IFNULL(p.p10c07,''))<=2 OR UPPER(IFNULL(p.p10c07,'')) IN ('NO','NA','N/A','POR','NADA','ESTUDIA','ESTUDIAR') ) AND 
	DATEDIFF(CURDATE(),(SELECT STR_TO_DATE(MAX(v.r1_fecha_inicial),'%d/%m/%y') FROM registro_de_visitas_pr v WHERE v.`level-1-id`=l.`level-1-id`))<=4;
"""

# Conectar a la base de datos MySQL
try:
    cnn = mysql.connector.connect(
        user='mchinchilla',
        password='mchinchilla$2023',
        host='20.10.8.4',
        port=3307,
        database='ENCOVI_PR'
    )
    cursor = cnn.cursor()

    # Obtener datos de ocupaciones
    data = pd.read_sql(sql, cnn)
    
except Exception as e:
    print(e)
finally:
    cnn.close()

# Procesar datos si hay al menos una fila
if data.shape[0] >= 1:
    ruta = "Mario"
    grupos = "GruposC2.xlsx"
    
    # Leer el archivo de grupos
    gs = pd.read_excel(grupos)
    
    # Unir con el DataFrame 'data'
    data = pd.merge(data, gs, on=['DEPTO', 'MUPIO', 'SECTOR'], how='left')
    
    print(f"Total de inconsistencias: {data.shape[0]}")
    
    # Crear carpeta si no existe
    if not os.path.exists("Mario"):
        os.mkdir("Mario")
        
    # Crear carpeta con marca temporal
    timestamp = pd.Timestamp.now().strftime("%d-%m-%H-%M")
    timestamp_folder = f"Mario/Inconsistencias_{timestamp}"
    os.mkdir(timestamp_folder)
    
    # Guardar el archivo Excel de inconsistencias totales
    data.to_excel(os.path.join(timestamp_folder, "Inconsistencias.xlsx"), index=False)
    
    # Guardar los archivos de inconsistencias por grupo
    lista = data['GRUPO'].unique()
    for item in lista:
        cuadro = data[data['GRUPO'] == item].drop(columns=['VALOR', 'GRUPO'])
        nombre = f"InconsistenciasGRUPO{item}.xlsx"
        
        print(f">> {nombre} -> {cuadro.shape[0]}")
        
        cuadro.to_excel(os.path.join(timestamp_folder, nombre), index=False)
else:
    print("No se recuperó información")


C:\Users\mchinchilla\AppData\Local\Temp\ipykernel_12064\3778974084.py:69: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql(sql, cnn)


Total de inconsistencias: 105
>> InconsistenciasGRUPO9.xlsx -> 4
>> InconsistenciasGRUPO31.xlsx -> 4
>> InconsistenciasGRUPO25.xlsx -> 8
>> InconsistenciasGRUPO38.xlsx -> 12
>> InconsistenciasGRUPO37.xlsx -> 9
>> InconsistenciasGRUPO11.xlsx -> 3
>> InconsistenciasGRUPO2.xlsx -> 8
>> InconsistenciasGRUPO30.xlsx -> 4
>> InconsistenciasGRUPO15.xlsx -> 4
>> InconsistenciasGRUPO32.xlsx -> 4
>> InconsistenciasGRUPO35.xlsx -> 7
>> InconsistenciasGRUPO4.xlsx -> 7
>> InconsistenciasGRUPO34.xlsx -> 5
>> InconsistenciasGRUPO8.xlsx -> 4
>> InconsistenciasGRUPO18.xlsx -> 8
>> InconsistenciasGRUPO29.xlsx -> 3
>> InconsistenciasGRUPO7.xlsx -> 3
>> InconsistenciasGRUPO27.xlsx -> 1
>> InconsistenciasGRUPO24.xlsx -> 4
>> InconsistenciasGRUPO14.xlsx -> 1
>> InconsistenciasGRUPO5.xlsx -> 2
